In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.extend([ '../', '../../'])
import torch
import torch.nn as nn
import torch.optim as optim
from data import input_data
import dataloader as dl
from args import Args
from model import Het_En, Classifier, EdgePredictor, Het_classify
from smote import oversample
from train import train_smote, test_smote

# Set device to GPU if available, else use CPU
args = Args()
args.aminer_train()
torch.cuda.empty_cache()

Current device: NVIDIA A100 80GB PCIe


In [2]:
# data = input_data(args) 
# torch.save(data, 'data.pt')
data = torch.load('../../data/aminer/am_data.pt')
print(data)

HeteroData(
  a={
    num_nodes=20171,
    y=[20171]
  },
  p={ num_nodes=13250 },
  v={ num_nodes=18 },
  p_title_embed={ x=[13250, 128] },
  p_abstract_embed={ x=[13250, 128] },
  p_net_embed={ x=[13250, 128] },
  p_a_net_embed={ x=[13250, 128] },
  p_p_net_embed={ x=[13250, 128] },
  p_v_net_embed={ x=[13250, 128] },
  a_net_embed={ x=[20171, 128] },
  a_text_embed={ x=[20171, 128] },
  v_net_embed={ x=[18, 128] },
  v_text_embed={ x=[18, 128] },
  (a, walk, a)={ edge_index=[2, 99480] },
  (a, walk, p)={ edge_index=[2, 86444] },
  (a, walk, v)={ edge_index=[2, 31716] },
  (p, walk, a)={ edge_index=[2, 65534] },
  (p, walk, p)={ edge_index=[2, 66174] },
  (p, walk, v)={ edge_index=[2, 20838] },
  (v, walk, a)={ edge_index=[2, 90] },
  (v, walk, p)={ edge_index=[2, 90] },
  (v, walk, v)={ edge_index=[2, 27] }
)


In [3]:
print(data['a','walk','p'].edge_index)
device = args.device
data = data.to(device)

# Send all x tensors to the device
data['p_title_embed']['x'] = data['p_title_embed']['x'].to(device)
data['p_abstract_embed']['x'] = data['p_abstract_embed']['x'].to(device)
data['p_net_embed']['x'] = data['p_net_embed']['x'].to(device)
data['p_a_net_embed']['x'] = data['p_a_net_embed']['x'].to(device)
data['p_p_net_embed']['x'] = data['p_p_net_embed']['x'].to(device)
data['p_v_net_embed']['x'] = data['p_v_net_embed']['x'].to(device)
data['a_net_embed']['x'] = data['a_net_embed']['x'].to(device)
data['a_text_embed']['x'] = data['a_text_embed']['x'].to(device)
data['v_net_embed']['x'] = data['v_net_embed']['x'].to(device)
data['v_text_embed']['x'] = data['v_text_embed']['x'].to(device)

# Send all y tensors to the device
data['a']['y'] = data['a']['y'].to(device)

# Send all edge_index tensors to the device
data['a', 'walk', 'a']['edge_index'] = data['a', 'walk', 'a']['edge_index'].to(device)
data['a', 'walk', 'p']['edge_index'] = data['a', 'walk', 'p']['edge_index'].to(device)
data['a', 'walk', 'v']['edge_index'] = data['a', 'walk', 'v']['edge_index'].to(device)
data['p', 'walk', 'a']['edge_index'] = data['p', 'walk', 'a']['edge_index'].to(device)
data['p', 'walk', 'p']['edge_index'] = data['p', 'walk', 'p']['edge_index'].to(device)
data['p', 'walk', 'v']['edge_index'] = data['p', 'walk', 'v']['edge_index'].to(device)
data['v', 'walk', 'a']['edge_index'] = data['v', 'walk', 'a']['edge_index'].to(device)
data['v', 'walk', 'p']['edge_index'] = data['v', 'walk', 'p']['edge_index'].to(device)
data['v', 'walk', 'v']['edge_index'] = data['v', 'walk', 'v']['edge_index'].to(device)

edge_indices = [ data['a', 'walk', 'a'].edge_index, data['a', 'walk', 'p'].edge_index, data['a', 'walk', 'v'].edge_index ]

tensor([[    0,     0,     0,  ..., 20170, 20170, 20170],
        [10523, 10442, 10329,  ...,  4378, 12456,  6612]])


In [4]:
c_train_num = dl.train_num(data['a'].y, args.im_class_num, args.class_sample_num, args.im_ratio)
print(c_train_num, sum(c_train_num))
train_idx, val_idx, test_idx, c_num_mat = dl.segregate(data['a'].y, c_train_num)
print("train_idx: ", train_idx, len(train_idx))
print("val_idx: ", val_idx, len(val_idx))
print("test_idx: ", test_idx, len(test_idx))
# print(c_num_mat)

[50, 20, 50, 50] 170
0 335
1 202
2 299
3 289
train_idx:  [11858, 1319, 6716, 12648, 18584, 9860, 9990, 13416, 17201, 5002, 11482, 3976, 19004, 3376, 12461, 13631, 12056, 5404, 15651, 11195, 16777, 3893, 9364, 17356, 13015, 2918, 9029, 116, 14009, 15679, 12115, 11980, 15168, 5506, 20168, 12878, 4170, 14682, 18939, 14004, 13759, 19549, 12696, 15060, 12589, 5042, 7909, 14304, 16946, 20024, 7858, 19514, 10387, 14825, 8024, 12172, 14642, 14133, 19529, 7015, 6228, 6710, 14024, 3535, 12795, 15569, 7481, 18560, 6769, 10930, 10953, 17965, 14867, 3262, 15304, 6884, 6133, 8272, 11659, 14878, 17446, 15876, 10086, 9938, 14011, 8595, 14683, 3230, 9180, 296, 5352, 14442, 2402, 15574, 15393, 641, 13386, 10865, 7484, 12161, 12309, 11061, 14312, 17692, 121, 5946, 6961, 7022, 8065, 13584, 8755, 10301, 1595, 8308, 6663, 12545, 7602, 8364, 5771, 6410, 14674, 651, 3842, 30, 13934, 16999, 13231, 950, 664, 17532, 5983, 15620, 18849, 9188, 13730, 17924, 13165, 14436, 59, 8711, 2755, 4095, 16477, 19809, 10196, 

In [5]:
# os_mode = 'up'
# portion = sum(c_train_num)/(4*c_train_num[1])
# if os_mode != 'reweight' and os_mode != 'no':
#     if portion >= 1:
#         args.node_dim[0] = args.A_n + (portion-1)*c_train_num[1] + int((portion - int(portion))*c_train_num[1])
#     else:
#         args.node_dim[0] = args.A_n + int((portion - int(portion))*c_train_num[1])

# args.node_dim[0] += 22

In [6]:
encoder = Het_En(args.embed_dim, args.dropout)
classifier = Het_classify(args.embed_dim, args.nclass, args.dropout)
decoder_a = EdgePredictor(args.embed_dim)
decoder_p = EdgePredictor(args.embed_dim)
decoder_v = EdgePredictor(args.embed_dim)
decoder_list = [decoder_a, decoder_p, decoder_v]
#print(features.shape)
encoder.to(device)
classifier.to(device)
for decoder in decoder_list:
    decoder.to(device)

## Training Part

In [7]:
torch.cuda.empty_cache()
train_smote(data, edge_indices, encoder, classifier, decoder_list, train_idx, val_idx, test_idx, args, os_mode = 'smote', train_mode = '')

Epoch [1/500], Loss: 1.3910, Accuracy: 0.1927, Edge Accuracy: []
Class 0: AUC-ROC- 0.2410, F1 Score- 0.0333; Class 1: AUC-ROC- 0.6725, F1 Score- 0.4375; Class 2: AUC-ROC- 0.4906, F1 Score- 0.0333; Class 3: AUC-ROC- 0.5751, F1 Score- 0.0000; Macro-Average AUC-ROC: 0.4948; Macro-Average F1 Score: 0.1260
Validation Loss: 1.3869, Validation Accuracy: 0.2050, Validation Edge Accuracy: []
Class 0: AUC-ROC- 0.2670, F1 Score- 0.0000; Class 1: AUC-ROC- 0.6629, F1 Score- 0.4606; Class 2: AUC-ROC- 0.5320, F1 Score- 0.0000; Class 3: AUC-ROC- 0.6689, F1 Score- 0.0000; Macro-Average AUC-ROC: 0.5327; Macro-Average F1 Score: 0.1152
Test Loss: 1.3871, Test Accuracy: 0.2025, Test Edge Accuracy: []
Class 0: AUC-ROC- 0.2921, F1 Score- 0.0000; Class 1: AUC-ROC- 0.6753, F1 Score- 0.4731; Class 2: AUC-ROC- 0.4952, F1 Score- 0.0000; Class 3: AUC-ROC- 0.6300, F1 Score- 0.0000; Macro-Average AUC-ROC: 0.5232; Macro-Average F1 Score: 0.1183

Epoch [2/500], Loss: 1.3873, Accuracy: 0.2083, Edge Accuracy: []
Class 0

In [8]:
test_smote(data, edge_indices, encoder, classifier, decoder_list, test_idx, args = args, dataset = 'Test', os_mode = 'no', train_mode = '')

Test Loss: 0.8805, Test Accuracy: 0.9625, Test Edge Accuracy: []
Class 0: AUC-ROC- 1.0000, F1 Score- 0.9804; Class 1: AUC-ROC- 0.9995, F1 Score- 0.9798; Class 2: AUC-ROC- 0.9778, F1 Score- 0.9320; Class 3: AUC-ROC- 0.9983, F1 Score- 0.9659; Macro-Average AUC-ROC: 0.9939; Macro-Average F1 Score: 0.9645


(0.9625, 0.9939, 0.9645206575371916)